In [ ]:
# !pip install pytube
# !pip install youtube-search-python
# !pip install --trusted-host pypi.python.org moviepy
# !pip install imageio-ffmpeg
# !pip install ytmusicapi
# !pip install ffmpeg
# !pip install imageio-ffmpeg

In [ ]:
from pytube import YouTube
from youtubesearchpython import VideosSearch
from tqdm import tqdm
import pandas as pd
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import os
import sqlite3
import json
from ytmusicapi import YTMusic
import imageio

In [ ]:
headers = #Provide headers to your youtube music
# Create Headers auth json file in this directory
YTMusic.setup(filepath="headers_auth.json", headers_raw=headers)
ytmusic = YTMusic('headers_auth.json')

In [ ]:
import subprocess

def get_duration(input_video):
    cmd = ["ffprobe", "-i", input_video, "-show_entries", "format=duration",
           "-v", "quiet", "-sexagesimal", "-of", "csv=p=0"]
    x = subprocess.check_output(cmd).decode("utf-8").strip()
    lis = x.split(":")
    seconds = 0
    seconds += float(lis[0])*3600
    seconds += float(lis[1])*60
    seconds += float(lis[2])
    return seconds

In [ ]:
filtered_triplet = pd.read_csv("filtered_triplets.csv") # Path to the triplets you want the songs of
print(filtered_triplet.shape)
filtered_triplet.head()
songid_set = set(filtered_triplet["song"])

In [ ]:
# cnx = sqlite3.connect('track_metadata.db')
# df = pd.read_sql_query("SELECT * FROM songs", cnx)
# dic = {}
# for index, row in tqdm(df.iterrows()):
#     if row["song_id"] in songid_set:
#         dic[row["song_id"]] = row["title"] + " " + row["artist_name"]
# a_file = open("song_search.json", "w")
# json.dump(dic, a_file)
# a_file.close()

In [ ]:
a_file = open("song_search.json", "r") # Song search is a dictionary with key as song_id and value as the search string (in our case song name with artist name)
dic = json.load(a_file)
a_file.close()
print(len(dic))

In [ ]:
temp_ids = sorted(list(dic.keys()))
dic_temp = {}
for i in temp_ids:
    dic_temp[i] = dic[i]

In [ ]:
error_indices=[]
for temp_id in tqdm(dic_temp, ncols = 100):
    try:
        path1="Songs/"+temp_id+"_1.mp4"
        path="Songs/"+temp_id+".mp4"
        search_results = ytmusic.search(dic[temp_id])
        yt = YouTube(ytmusic.get_song(search_results[0]["videoId"])["microformat"]["microformatDataRenderer"]["iosAppArguments"])
        
        yt.streams.get_audio_only().download(output_path='Songs',filename=temp_id+"_1.mp4")
        sec = get_duration(path1)
        if sec > 90:
            ffmpeg_extract_subclip(path1, 60, 90, targetname=path)
        elif sec <= 30:
            # os.remove(path1)
            error_indices.append(temp_id)
            with open("error_log.txt", "a") as fp: 
                fp.write(str(temp_id)+"\n")
        elif sec > 60:
            ffmpeg_extract_subclip(path1, 30, 60, targetname=path)
        else:
            ffmpeg_extract_subclip(path1, int(sec)-30, int(sec), targetname=path)
        os.remove(path1)
    except:
        error_indices.append(temp_id)
        with open("error_log.txt", "a") as fp: 
            fp.write(str(temp_id)+"\n")